In [ ]:
##luigi-vars
RAW_HD5 = 'default'

In [ ]:
import vcfnp
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import allel
import seaborn as sns
import pandas as pd
from sklearn.mixture import GaussianMixture
sns.set_style('white')
%matplotlib inline

In [ ]:
callset = h5py.File(RAW_HD5, mode='r')
calldata = callset['calldata']
genotypes = allel.GenotypeChunkedArray(callset['calldata']['genotype'])
samples = list(callset['samples'])
variants = allel.VariantChunkedTable(callset['variants'])[:]
df_variants = pd.DataFrame(np.array(variants))
df_variants = df_variants.drop(df_variants.columns[np.all(df_variants == 0, axis=0)],axis=1)

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.cubehelix_palette(n_colors=24, as_cmap=True, dark=0, light=1)
    cmap.set_under('white')
    cmap.set_bad('white')
    plt.hexbin(x, y, gridsize=40, cmap=cmap, **kwargs)

def hist(x, **kwargs):
    sns.distplot(x, kde=False)
    
df_variants['logDP'] = np.log1p(df_variants['DP'])
df_variants['SOR_clip'] = np.clip(df_variants['SOR'],a_max=6, a_min=0)
df_variants['ReadPosRankSum_clip'] = np.clip(df_variants['ReadPosRankSum'],a_max=5, a_min=-5)


g = sns.PairGrid(data=df_variants[df_variants.is_snp], 
                 x_vars=['QD', 'logDP', 'AF', 'SOR_clip', 'ReadPosRankSum_clip'], 
                 y_vars=['QD', 'logDP', 'AF', 'SOR_clip', 'ReadPosRankSum_clip'],
                 size=4,
                 diag_sharey=False)
g.map_diag(hist)
g.map_lower(hexbin)

In [ ]:
tot_gq = np.sum(calldata['GQ'][variants.is_snp,:], axis=1)
tot_rgq = np.sum(calldata['RGQ'][~variants.is_snp,:], axis=1)

fig, ax = plt.subplots(ncols=1,nrows=1)
sns.distplot(tot_gq/variants[variants.is_snp]['DP'], kde=False, ax=ax[0])
sns.distplot(tot_rgq/variants[~variants.is_snp]['DP'], kde=False, ax=ax[1])

In [ ]:
from collections import Counter
P_dp = np.zeros((np.max(variants['DP'])+1))
dp_count = Counter(variants['DP'])
for k,v in dp_count.items():
    P_dp[k] = v

P_dp /= len(variants['DP'])
plt.plot(P_dp, '.')
plt.loglog()
